# Split (분할)
![rag_split](figures/rag_split.png)

- Load 한 문서를 지정한 기준의 덩어리(chunk)로 나눈다.
- LLM에 전체 내용을 다보는 것은 비효율 적이다. 그래서 문서를 적당한 size로 나눈 뒤 질문과 연관있는 chunk만 전송하도록 한다.
  - 로딩한 문서에서 질문과 관련되어 참조할 내용들을 찾아서 LLM 모델에 전송하는 것이 목적이다. 그런데 내용이 너무 크면 질문과 관련없는 내용들도 같이 전송되게 된다. 또한 OpenAI와 같은 폐쇄형 LLM에 요청할 경우 비용이 증가의 원인이 된다. 

## 주요 Spliter
- https://api.python.langchain.com/en/latest/text_splitters_api_reference.html

### CharacterTextSplitter
가장  기본적은 Text spliter
- 한 개의 구분자를 기준으로 분리한다.
- chunk size: 분리된 chunk의 최대 글자수를 지정한다.
- overlap 기능이 있어 이전 chunk의 뒷부분이 시작 부분에 겹쳐 나오도록해서 context가 애매하게 끝나지 않게 한다.  
- 주요 파라미터
    - chunk_size: chunk 최대 길이를 지정.
    - chunk_overlap: chunk 간의 겹치는 문자 수를 설정
    - seperator: 구분 문자열을 지정. 기본값: "\n\n"

### RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 긴 텍스트를 가장 효율적으로 분할한다.
- 여러 구분자를 순차적으로 적용하여 각 chunk가 지정된 최대 길이를 초과하지 않도록 분할한다.
- 분할 기준 문자
    1. 두 개의 줄바꿈 문자 ("\n\n")
    2. 한 개의 줄바꿈 문자 ("\n")
    3. 공백 문자 (" ")
    4. 빈 문자열 ("")
- 방식
    1. '\n\n' (엔터두개) 를 기준으로 분할 한다.
    2. 분할 된 조각(chunk)이 지정한  chunk 크기보다 클 경우 다음 분할 구분 문자인 '\n' 으로 다시 분할 한다.
    3. 그래도 chunk size를 초과하면 공백문자로 분할하고 그래도 크면 다음 기준 문자인 빈문자로 분할 해 지정한 chunk size를 넘지 않도록 한다.       
- 주요 파라미터
    - chunk_size: chunk 최대 길이를 지정.
    - chunk_overlap: chunk 간의 겹치는 문자 수를 설정 이를 통해 문맥이 중간에 끊기지 않도록 한다.
    - separator**s**: 기본 분할 구분 문자 이외의 것들을 추가적으로 지정할 수있다.

## Token 수 기준으로 나누기

- LLM 언어 모델들은 입력 토큰 수 제한이 있어서 요청시 제한 토큰수 이상의 프롬프트는 전송할 수 없다.
- 따라서 텍스트를 chunk로 분할할 때는 글자수 보다 **토큰 수를 기준으로 크기를 지정하는 것**이 좋다.  
- 토큰기반 분할은 텍스트의 의미를 유지하면서 분할하는 방식이므로 문자 기반 분할과 같이 단어가 중간잘리는 것들을 방지할 수 있다. 
- 토큰 수 계산할 때는 사용하는 언어 모델에 사용된 것과 동일한 tokenizer를 사용하는 것이 좋다.
  - 예를 들어 OpenAI의 GPT 모델을 사용할 경우 tiktoken 라이브러리를 활용하여 토큰 수를 정확하게 계산할 수 있다.

### [tiktoken](https://github.com/openai/tiktoken) tokenizer 기반 분할
- OpenAI에서 GPT 모델을 학습할 때 사용한 `BPE` 방식의 tokenizer. **OpenAI 언어모델을 사용할 경우 이것을 사용하는 것이 좀 더 정확하게 토큰계산할 수있다.**
- Splitter.from_tiktoken_encoder() 메소드를 이용해 생성
  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder()`
  - `CharacterTextSplitter.from_tiktoken_encoder()`
- 파라미터
  - encode_name: 인코딩 방식(토큰화 규칙)을 지정. OpenAI는 GPT 모델들 마다 다른 방식을 사용했다. 그래서 사용하려는 모델에 맞는 인코딩 방식을 지정해야 한다.
    - `cl100k_base`: GPT-4 및 GPT-3.5-Turbo 모델에서 사용된 방식.
    - `r50k_base:` GPT-3 모델에서 사용된 방식 
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- tiktoken 설치
  - `pip install tiktoken`

### HuggingFace Tokenizer
- HuggingFace 모델을 사용할 경우 그 모델이 사용한 tokenizer를 이용해 토큰 기반으로 분할 할 수있다.
- `from_huggingface_tokenizer()` 메소드를 이용해 생성
- 파라미터
  - tokenizer: HuggingFace tokenizer 객체
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- `transformers` 라이브러리를 설치해야 한다.
  - `pip install transformers` 